In [1]:
import torch
import os
import math
import numpy as np
from PIL import Image
import PIL
import sys
sys.path.insert(0, '/datadrive/AACS2023')
from fdf256dataset import FDF256Dataset

Could not load pyspng. Defaulting to pillow image backend.


In [3]:
dataset_path = '/datadrive/FDF/dataset/val'
dataset = FDF256Dataset(dirpath=dataset_path, load_keypoints=True, transform=None, load_impath=True)

Dataset loaded from: /datadrive/FDF/dataset/val. Number of samples:6531


In [7]:
pred_path = '/datadrive/AACS2023/HAT/HAT/results/HAT_SRx4_ImageNet-LR/visualization/custom/'

In [8]:
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf


total_psnr = 0.0
total_ssim = 0.0

for i in tqdm(range(len(dataset))):
    original_img = dataset[i]["img"]
    img_name: str = str(dataset[i]['impath']).split('/')[-1]
    img_idx = img_name.split('.')[0]
    
    pred_img_path = os.path.join(pred_path, f'{img_idx}_HAT_SRx4_ImageNet-LR.png')
    pred_img = Image.open(pred_img_path)
    pred_img = np.asarray(pred_img)
    
    im_psnr: float = tf.image.psnr(pred_img, original_img, max_val=255)
    im_ssim: float = ssim(im1=original_img, im2=pred_img, data_range=pred_img.max() - pred_img.min(), channel_axis=2)

    total_psnr += im_psnr
    total_ssim += im_ssim
    
avg_psnr = total_psnr / len(dataset)
avg_ssim = total_ssim / len(dataset)

print(f'Average PSNR = {avg_psnr}')
print(f'Average SSIM = {avg_ssim}')


2023-04-27 15:50:54.229697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 15:50:58.332239: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
  0%|          | 0/6531 [00:00<?, ?it/s]2023-04-27 15:51:15.628523: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14087 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0
2023-04-27 15:51:15.633619: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14087 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0002:00:00.0, compute capability: 7.0
2023

Average PSNR = 34.084320068359375
Average SSIM = 0.9206088639405688
